In [ ]:
#math library for writing sq.rt
import math

In [ ]:
# General imports
import numpy as np

# Pre-defined ansatz circuit and operator class for Hamiltonian
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp

# SciPy minimizer routine
from scipy.optimize import minimize

# Plotting functions
import matplotlib.pyplot as plt

In [ ]:
# runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator

#AerSimulator
from qiskit_aer import AerSimulator
from scipy.optimize import basinhopping

service = QiskitRuntimeService(channel='###')
#service.backends(simulator=True, operational=True, min_num_qubits=5)
#backend = service.least_busy(min_num_qubits=127)
#print(backend)

#simulator
backend = AerSimulator()


In [ ]:
l01 = math.sqrt(2)
l02 = math.sqrt(10)
l03 = 3
l12 = 2
l13 = math.sqrt(5)
l23 = 1
g = 20

In [ ]:
H0 = SparsePauliOp.from_sparse_list([("", [], l01), ("", [], l02), ("", [], l03), ("", [], l12), ("", [], l13), ("", [], l23), ("Z", [0], l01/4), ("Z", [0], l02/4), ("Z", [0], l03/2), ("Z", [1], l01/2), ("Z", [1], l02/2), ("Z", [2], l01/4), ("Z", [2], l02/4), ("Z", [2], l03/2), ("Z", [3], l01/4), ("Z", [3], l12/4), ("Z", [3], l13/2),  ("ZZ", [1,3], l01/4), ("Z", [4], l01/2), ("Z", [4], l12/2),  ("ZZ", [0,4], l01/4), ("ZZ", [2,4], l01/4), ("Z", [5], l01/4), ("Z", [5], l12/4), ("Z", [5], l13/2),("ZZ", [1,5], l01/4), ("Z", [6], l02/4), ("Z", [6], l12/4), ("Z", [6], l23/2), ("ZZ", [1,6], l02/4),  ("ZZ", [4,6], l12/4), ("Z", [7], l02/2), ("Z", [7], l12/2),  ("ZZ", [0,7], l02/4), ("ZZ", [2,7], l02/4), ("ZZ", [3,7], l12/4), ("ZZ", [5,7], l12/4), ("Z", [8], l02/4), ("Z", [8], l12/4), ("Z", [8], l23/2), ("ZZ", [1,8], l02/4), ("ZZ", [4,8], l12/4), ("", [], (3/2)*g), ("Z", [0], g), ("", [], g/2), ("Z", [1], g), ("ZZ", [0,1], g/2), ("", [], g/2), ("Z", [2], g), ("ZZ", [0,2], g/2), ("ZZ", [1,2], g/2), ("", [], g/2), ("Z", [3], g), ("ZZ", [0,3], g/2), ("", [], g/2), ("Z", [4], g),  ("ZZ", [1,4], g/2), ("ZZ", [3,4], g/2), ("", [], g/2), ("Z", [5], g), ("ZZ", [2,5], g/2), ("ZZ", [3,5], g/2), ("ZZ", [4,5], g/2), ("", [], g/2), ("Z", [6], g), ("ZZ", [0,6], g/2), ("ZZ", [3,6], g/2), ("", [], g/2), ("Z", [7], g), ("ZZ", [1, 7], g/2), ("ZZ", [4,7], g/2), ("ZZ", [6,7], g/2), ("", [], g/2), ("Z", [8], g), ("ZZ", [2,8], g/2), ("ZZ", [5,8], g/2), ("ZZ", [6,8], g/2), ("ZZ", [7,8], g/2), ("", [], g/2) ],  num_qubits=9)

In [ ]:
H0

In [ ]:
ansatz = ansatz = EfficientSU2(H0.num_qubits, reps=4)
ansatz.decompose().draw("mpl", style="iqp")

In [ ]:
num_params = ansatz.num_parameters
num_params

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=1)

ansatz_isa = pm.run(ansatz)

In [ ]:
#ansatz_isa.draw(output="mpl", idle_wires=False, style="iqp")

In [ ]:
hamiltonian_isa = H0.apply_layout(layout=ansatz_isa.layout)

In [ ]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")

    return energy

In [ ]:
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

In [ ]:
x0 = 2 * np.pi * np.random.random(num_params)

In [ ]:
x0

In [ ]:
with Session(backend=backend) as session:
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 10000

    res = minimize(
        cost_func,
        x0,
        args=(ansatz_isa, hamiltonian_isa, estimator),
        method="cobyla", options={'rhobeg': 4, 'tol': 1e-5, 'maxiter': 800}
    )

In [ ]:
print(session.details())

In [ ]:
res

In [ ]:
res.x

In [ ]:
all(cost_history_dict["prev_vector"] == res.x)

In [ ]:
cost_history_dict["iters"] == res.nfev

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(cost_history_dict["iters"]), cost_history_dict["cost_history"])
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost")
plt.draw()

In [ ]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Assign solution parameters to ansatz
qc = ansatz_isa.assign_parameters(res.x)
# Add measurements to our circuit
qc.measure_all()
pub= (qc, )
job = sampler.run([pub], shots=int(1e4))
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val/shots for key, val in counts_int.items()}
final_distribution_bin = {key: val/shots for key, val in counts_bin.items()}
#print(final_distribution_int)


In [ ]:
# auxiliary functions to sample most likely bitstring
def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]

keys = list(final_distribution_int.keys())
values = list(final_distribution_int.values())
most_likely = keys[np.argmax(np.abs(values))]
most_likely_bitstring = to_bitstring(most_likely, H0.num_qubits)
most_likely_bitstring.reverse()

print("Result bitstring:", most_likely_bitstring)

In [ ]:
# Sort the dictionary items by values in descending order
sorted_items = sorted(counts_bin.items(), key=lambda x: x[1], reverse=True)
# Display the sorted key-value pairs
for key, value in sorted_items:
    print(f'{key}: {value}')